# Detection visage

In [1]:
import ultralytics
from ultralytics import YOLO

ultralytics.checks()

Ultralytics YOLOv8.2.34 🚀 Python-3.12.3 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 5919MiB)
Setup complete ✅ (24 CPUs, 31.3 GB RAM, 356.6/915.3 GB disk)


## Création du modèle

On récupère un modèle très léger de detection fourni par ultralytics

In [2]:
# model = YOLO("./resources/yolov8n.pt")

## Entraînement

Entraînement du modèle sur un dataset spécifiquement annoté pour la detection de visage via des bounding boxes.

Le fichier "dataset.yaml" permet d'indiquer au modèle où se trouve les images d'entraînement, de validation et de test. Ainsi que les classes à detecter. Ici le but est de simplement detecter un visage humain.

In [5]:
 # Train the model
# results = model.train(data="./resources/visages/dataset.yaml", epochs=10, imgsz=640)

## Test du modèle

On récupère le meilleur modèle de notre précedent entraînement.

In [2]:
model = YOLO("./runs/detect/train11/weights/best.pt")

On teste le modèle sur une image

In [6]:
results = model('./resources/visages/test/frame_1572.jpg', visualize=True)

In [4]:
from PIL import Image
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Show results to screen (in supported environments)
    r.show()

On test le modèle sur une vidéo. 

A noter qu'il est préférable d'effectuer le test sur une petite vidéo, sinon cela demande pas mal de mémoire vive.

In [2]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("./runs/detect/train11/weights/best.pt")

# Open the video file
video_path = "./resources/visages/test/video.avi"

results = model.predict(source=video_path, save=True)


## Coupler la detection avec un classifieur

Après avoir un modèle qui détecte les visages humain, l'idée est de déterminer quel est le personnage de fiction correspondant au visage détecter. Sans partir dans le domaine de la reconnaissance faciale, l'idée principale est simplement de pouvoir attribuer une classe plus précise à un objet détecté.

Pour l'expérience, j'ai fait tourner le modèle de detection de visage sur une video d'un épisode de série. Toutes les box détectés par le modèle, j'en ai fait des images.

In [4]:
cap = cv2.VideoCapture("./resources/video2.avi")
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print( length )

i = 0
n = 0
while cap.isOpened():
    # Read video capture
    ret, frame = cap.read()

    results = model(frame, stream=True)
    # coordinates
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

            cropped_image = frame[y1:y2,x1:x2]

            # cv2.imshow("cropped", cropped_image)
            cv2.imwrite(f"./resources/all-visages/i-{n}.jpg", cropped_image)
            n+=1    
        
    i+=1
    print(i, "/", length)

Cela a généré plus de 60000 images. Pour classer toutes ces images en 3 classes : Eliot, Olivia et Inconnu j'ai utilisé un algorythme de Clustering (ou partitionnement des données) disponible dans le fichier kmeans.py. 

Cela va créer 50 répertoires avec un regroupement d'images similaires, cela permet d'aller beaucoup plus vite pour le tri des 60000 images.

Une fois les images séparées en 3 classes, on créé un classifieur standard que l'on enraîne avec YOLO.

Maintenant que le classifieur est prêt on peut l'utiliser en complément de notre modèle de detection pour identifier chaque box.

In [ ]:
model_classifier_visage = YOLO('./model-classifier-visage.pt')
model = YOLO("./runs/detect/train-v8n/weights/best.pt")

cap = cv2.VideoCapture("./resources/video2.avi")
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

i = 0
n = 0
while cap.isOpened():
    # Read video capture
    ret, frame = cap.read()

    results = model(frame, stream=True)
    # coordinates
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

            cropped_image = frame[y1:y2,x1:x2]

            name_visage = ""
            if cropped_image is not None:
                results = model_classifier_visage(cropped_image)
                for r in results:
                    name_visage = model_classifier_visage.names[r.probs.top1]

            # put box in cam
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 1)
            org = [x1+5, y1+15]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 0.5
            color = (255, 255, 255)
            thickness = 1

            cv2.putText(frame, str(name_visage), org, font, fontScale, color, thickness)
            n+=1

    # Display each frame
    cv2.imshow("video", frame)
    # show one frame at a time
    key = cv2.waitKey(0)
    while key not in [ord('q'), ord('k')]:
        key = cv2.waitKey(0)
    # Quit when 'q' is pressed
    if key == ord('q'):
        break
    i+=1
    print(i, "/", length)

## Exemples

<img src="./examples/01.png" width="500" />
<img src="./examples/02.png" width="500" />
<img src="./examples/03.png" width="500" />
<img src="./examples/04.png" width="500" />
<img src="./examples/05.png" width="500" />
<img src="./examples/06.png" width="500" />
<img src="./examples/07.png" width="500" />
<img src="./examples/08.png" width="500" />
<img src="./examples/09.png" width="500" />